In [2]:
import pandas as pd

import numpy as np

import string

from nltk.stem import PorterStemmer

from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords
from statsmodels.graphics.mosaicplot import mosaic

import matplotlib.pyplot as plt

import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

2025-07-27 15:37:38.186904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753630658.535999      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753630658.638462      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login

login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [18]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [19]:
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device="cuda",
)

Device set to use cuda


In [30]:
test_df4 = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type4_final.csv',index_col=0)
test_df4

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Liam believes that his wind tunnel is nearby.,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Attitude Premise
1,If Liam is an aerospace engineer and he tests ...,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Conjunction Premise
2,"Either Liam isn't an aerospace engineer, or he...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Disjunction Premise
3,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
4,Bill believes that his content management appl...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Attitude Premise
...,...,...,...,...,...,...,...,...
391,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome,premise
392,Aliya believes that her scrub is in the locker...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Attitude Premise
393,If Aliya is a nurse and she wears her scrub to...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Conjunction Premise
394,"Either Aliya isn't a nurse, or she wears her s...",Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Disjunction Premise


In [31]:
test_df5a = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type5a_final.csv',index_col=0)
test_df5a

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Jessica believes she'll watch a movie by Nolan...,Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,movie by Nolan again,Attitude Premise
1,If Jessica attends the conference and she neve...,Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,movie by Nolan again,Conjunction Premise
2,"Either Jessica doesn't attend the conference, ...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,movie by Nolan again,Disjunction Premise
3,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,movie by Nolan again,premise
4,Emily believes she'll read a book by Dan Brown...,Emily has read a book by Dan Brown before.,E,possessive,1,type5_again_2_01,book by Dan Brown before,Attitude Premise
...,...,...,...,...,...,...,...,...
1595,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,a moment in peace again,premise
1596,Zoey believes that she'll hold a student respo...,Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Attitude Premise
1597,If Zoey decides to drive and she never holds a...,Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Conjunction Premise
1598,"Either Zoey doesn't decide to drive, or she an...",Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Disjunction Premise


In [32]:
test_df5p = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type5p_final.csv',index_col=0)
test_df5p

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Jessica believes that her friend is upstairs.,Jessica has a friend.,E,possessive,0,type5_possessive_1_00,her friend,Attitude Premise
1,If Jessica attends the conference and her frie...,Jessica has a friend.,E,possessive,0,type5_possessive_1_00,her friend,Conjunction Premise
2,"Either Sarah doesn't attend the conference, or...",Jessica has a friend.,E,possessive,0,type5_possessive_1_00,her friend,Disjunction Premise
3,"If Jessica attends the conference, her colleag...",Jessica has a friend.,E,possessive,0,type5_possessive_1_00,her friend,premise
4,Emily believes her best friend is nearby.,Emily has a best friend.,E,possessive,1,type5_possessive_1_01,her best friend,Attitude Premise
...,...,...,...,...,...,...,...,...
1595,"If Betty misses the bus, her brother will driv...",Betty has a brother.,E,possessive,398,type5_possessive_1_398,her brother,premise
1596,Zoey believes that her friends are at the conc...,Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Attitude Premise
1597,If Zoey decides to drive and her friends meet ...,Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Conjunction Premise
1598,"Either Zoey doesn't decide to drive, or she an...",Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Disjunction Premise


In [33]:
test_df = pd.concat([test_df4,test_df5a,test_df5p])

In [26]:
def classify_pair(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: {premise}
        Hypothesis: {hypothesis}
        Output:
        '''},
    ]
    
    outputs = llama_pipeline(
        messages,
        max_new_tokens=1,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    
    return outputs[0]["generated_text"][-1]['content'][0]

In [14]:
import warnings
warnings.filterwarnings("ignore")  # Suppresses all warnings

In [35]:
test_df['Predicted'] = test_df.apply(classify_pair, axis=1)
test_df.to_csv('llama_part1_subset2_results.csv')

In [30]:
test_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type,Predicted
0,"If Alexander is an electrical technician, John...",Alexander has an insulation tester.,N,possessive,0,type4_possessive_00,John's friend,unrelated,C
1,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0,type4_possessive_00,his insulation tester,related,E
2,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,N,possessive,0,type4_possessive_00,his wire connectors,somewhat_related,C
3,"If Ali is a firefighter, John's friend will bo...",Ali has a firehose.,N,possessive,1,type4_possessive_01,John's friend,unrelated,C
4,"If Ali is a firefighter, he'll borrow us his f...",Ali has a firehose.,E,possessive,1,type4_possessive_01,his firehose,related,E
...,...,...,...,...,...,...,...,...,...
1187,"If Zoey passes the exam, her guardians will ta...",Zoey has parents.,N,possessive,398,type5P_possessive_398,her guardians,somewhat_related,C
1188,"If Zoey passes the exam, her parents will take...",Zoey has parents.,E,possessive,398,type5P_possessive_398,her parents,related,C
1189,"If Zoey takes the train to Boston, Tina's frie...",Zoey has a brother.,N,possessive,399,type5P_possessive_399,Tina's friend,unrelated,N
1190,"If Zoey takes the train to Boston, his brother...",Zoey has a brother.,E,possessive,399,type5P_possessive_399,his brother,related,C


In [36]:
test_df['Predicted'].value_counts()

Predicted
E    2868
C     453
N     143
S      78
I      50
#       4
Name: count, dtype: int64

In [3]:
df = pd.read_csv('/kaggle/input/confer-extension-results/Part1/llama_part1_subset3_results.csv')
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Unnamed: 0,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type,predicted_label,expected_label
0,0.0,"If Alexander is an electrical technician, John...",Alexander has an insulation tester.,N,possessive,0.0,type4_possessive_00,John's friend,unrelated,E,N
1,1.0,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0.0,type4_possessive_00,his insulation tester,related,E,N
2,2.0,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,N,possessive,0.0,type4_possessive_00,his wire connectors,somewhat_related,E,N
3,3.0,"If Ali is a firefighter, John's friend will bo...",Ali has a firehose.,N,possessive,1.0,type4_possessive_01,John's friend,unrelated,E,N
4,4.0,"If Ali is a firefighter, he'll borrow us his f...",Ali has a firehose.,E,possessive,1.0,type4_possessive_01,his firehose,related,E,N
...,...,...,...,...,...,...,...,...,...,...,...
2694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df= df.dropna()
df

,Unnamed: 0,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type,predicted_label,expected_label
0,0.0,"If Alexander is an electrical technician, John...",Alexander has an insulation tester.,N,possessive,0.0,type4_possessive_00,John's friend,unrelated,E,N
1,1.0,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0.0,type4_possessive_00,his insulation tester,related,E,N
2,2.0,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,N,possessive,0.0,type4_possessive_00,his wire connectors,somewhat_related,E,N
3,3.0,"If Ali is a firefighter, John's friend will bo...",Ali has a firehose.,N,possessive,1.0,type4_possessive_01,John's friend,unrelated,E,N
4,4.0,"If Ali is a firefighter, he'll borrow us his f...",Ali has a firehose.,E,possessive,1.0,type4_possessive_01,his firehose,related,E,N
...,...,...,...,...,...,...,...,...,...,...,...
2292,795.0,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,397.0,type5p_possessive_397,his wife,unrelated,E,N
2293,796.0,"If Betty misses the train, her brother will dr...",Betty has a brother.,E,possessive,398.0,type5p_possessive_398,her brother,related,E,N
2294,797.0,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,398.0,type5p_possessive_398,her brother,unrelated,E,N
2295,798.0,"If Zoey decides to bike, her friends will meet...",Zoey has friends.,E,possessive,399.0,type5p_possessive_399,her friends,related,E,N


In [5]:
orig_df = df[df['premise_type'] == 'related']
orig_df

,Unnamed: 0,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type,predicted_label,expected_label
1,1.0,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0.0,type4_possessive_00,his insulation tester,related,E,N
4,4.0,"If Ali is a firefighter, he'll borrow us his f...",Ali has a firehose.,E,possessive,1.0,type4_possessive_01,his firehose,related,E,N
7,7.0,"If Alice is a midwife, she'll travel with her ...",Alice has a birthing kit.,E,possessive,2.0,type4_possessive_02,her birthing kit,related,E,N
10,10.0,"If Aliya is a nurse, she'll wear her scrub to ...",Aliya has scrubs.,E,possessive,3.0,type4_possessive_03,her scrub,related,E,N
13,13.0,"If Amelia is a seamstress, she'll fix her sewi...",Amelia has a sewing machine.,E,possessive,4.0,type4_possessive_04,her sewing machine,related,C,N
...,...,...,...,...,...,...,...,...,...,...,...
2287,790.0,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,395.0,type5p_possessive_395,her brother,related,E,N
2289,792.0,"If Susan finishes his assignment early, his co...",Susan has a colleague.,E,possessive,396.0,type5p_possessive_396,his colleague,related,N,N
2291,794.0,"If Margaret finishes work early, his wife will...",Margaret has a wife.,E,possessive,397.0,type5p_possessive_397,his wife,related,E,N
2293,796.0,"If Betty misses the train, her brother will dr...",Betty has a brother.,E,possessive,398.0,type5p_possessive_398,her brother,related,E,N


In [6]:
new_df = df[df['premise_type'] == 'related']
new_df

,Unnamed: 0,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type,predicted_label,expected_label
1,1.0,"If Alexander is an electrical technician, he'l...",Alexander has an insulation tester.,E,possessive,0.0,type4_possessive_00,his insulation tester,related,E,N
4,4.0,"If Ali is a firefighter, he'll borrow us his f...",Ali has a firehose.,E,possessive,1.0,type4_possessive_01,his firehose,related,E,N
7,7.0,"If Alice is a midwife, she'll travel with her ...",Alice has a birthing kit.,E,possessive,2.0,type4_possessive_02,her birthing kit,related,E,N
10,10.0,"If Aliya is a nurse, she'll wear her scrub to ...",Aliya has scrubs.,E,possessive,3.0,type4_possessive_03,her scrub,related,E,N
13,13.0,"If Amelia is a seamstress, she'll fix her sewi...",Amelia has a sewing machine.,E,possessive,4.0,type4_possessive_04,her sewing machine,related,C,N
...,...,...,...,...,...,...,...,...,...,...,...
2287,790.0,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,395.0,type5p_possessive_395,her brother,related,E,N
2289,792.0,"If Susan finishes his assignment early, his co...",Susan has a colleague.,E,possessive,396.0,type5p_possessive_396,his colleague,related,N,N
2291,794.0,"If Margaret finishes work early, his wife will...",Margaret has a wife.,E,possessive,397.0,type5p_possessive_397,his wife,related,E,N
2293,796.0,"If Betty misses the train, her brother will dr...",Betty has a brother.,E,possessive,398.0,type5p_possessive_398,her brother,related,E,N
